In [1]:
import compyler as cp

# Create a FHE program object
program = cp.FheProgram(scheme="CKKS", devices=[1]) # FheProgram 여기부터 plan.py의 FheProgram Class를 부르는것부터 시작임. 


In [2]:
#program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
b = -a + 2
c = a - b
program.cppcompile(c)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


In [12]:

# Add secret variables. This is a ciphernode.
a = program.add_secret("a", 40) 


In [ ]:
# Create a FHE circuit
b = -a + 2
cpp_code="""
Ciphertext fun(Ciphertext a) {
    // Create FHE circuit
    auto b = -a + 2;
    auto c = a - b;

    return b;
}"""

## auto, ;, // 삭제. 
# {}는 분석 후에 삭제. 
# Ciphertext a -> add_secret("a", 40)로 대체.
# scale = 40이라는 정보를 C++ 스럽게 집어넣을 방법이 없음. 
# return b; -> program.cppcompile(b)로 대체.

# 비정상적인 CPP 코드일 경우 에러 나와야 하므로, 제대로 된 CPP 파서로 체크 필요. 


In [14]:

program.cppcompile(b)


Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


In [2]:
from compyler.cipher_node import CipherNode

In [ ]:

cpp_code = """
Ciphertext a;
auto b = -a + 2;
"""

vars = parse_cpp_code(cpp_code)
print(vars)

In [3]:
cpp_code = """
Ciphertext fun(Ciphertext a) {
    // Create FHE circuit
    auto b = -a + 2;
    auto c = a - b;

    return c;
}
"""

# Function to parse and translate C++ code to Python
def cpp_to_python(cpp_code):
    cpp_lines = cpp_code.strip().splitlines()
    python_code = ""

    for line in cpp_lines:
        line = line.strip()
        if line.startswith("Ciphertext fun("):
            # Translate function declaration to adding secret
            func_signature = line.split("(")[1].split(")")[0].strip()
            var_name = func_signature.split()[1]
            python_code += f'a = program.add_secret("{var_name}", 40)\n'
        elif line.startswith("auto "):
            # Translate auto assignment to Python
            var_name, expression = line[5:].split(" = ")
            expression = expression.strip()[:-1]  # Remove the semicolon
            python_code += f"{var_name.strip()} = {expression}\n"
        elif line.startswith("return "):
            # Translate return statement to program.cppcompile
            return_var = line.split()[1][:-1]  # Get the variable name, remove semicolon
            python_code += f"program.cppcompile({return_var})\n"

    return python_code

# Convert the C++ code to Python
python_code = cpp_to_python(cpp_code)

# Add the Program initialization
full_python_code = """
program = cp.FheProgram(scheme="CKKS", devices=[1])
""" + python_code

# Print the full Python code to check
print("Full Python code:")
print(full_python_code)

# Create a dictionary to capture the local variables
local_vars = {}

# Execute the full Python code with the local_vars dictionary
exec(full_python_code)#, {}, local_vars)

Full Python code:

program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
b = -a + 2
c = a - b
program.cppcompile(c)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


In [15]:
def cpp_to_python(cpp_code):
    # Split the code into lines and initialize an empty string for Python code
    cpp_lines = cpp_code.strip().splitlines()
    python_code = ""
    
    for line in cpp_lines:
        line = line.strip()
        if line.startswith("Ciphertext "):
            # Translate variable declaration to Python with arguments
            var_name = line.split()[1][:-1]  # Remove the semicolon
            python_code += f'{var_name} = CipherNode("{var_name}", 40)\n'
        elif line.startswith("auto "):
            # Translate auto assignment to Python
            var_name, expression = line[5:].split(" = ")
            expression = expression.strip()[:-1]  # Remove the semicolon
            expression = expression.replace("a", "a")  # Directly mapping for simplicity
            python_code += f"{var_name} = {expression}\n"
    
    return python_code


partial_code = """
auto b = -a + 2;"""

# Convert the C++ code to Python
python_code = cpp_to_python(partial_code)

code_to_run = """
program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40) 
"""+python_code+"""
program.cppcompile(b)
"""
# Print the converted Python code
print("Converted Python code:")
print(code_to_run)

# Execute the Python code
exec(code_to_run)

Converted Python code:

program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40) 
b = -a + 2

program.cppcompile(b)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


In [3]:
import networkx as nx
from networkx.readwrite import json_graph
import json


def node2dict(node):
    fields = dict(vars(node))
    fields.pop("children")
    fields.pop("parents")
    return fields


def node2nxgraph(root):
    nodenum = 1
    node2num = {root: nodenum}
    rootdict = node2dict(root)
    rootdict["type"] = str(type(root))
    rootdict["root"] = True

    G = nx.DiGraph() # directed graph
    G.add_node(node2num[root], **rootdict)

    # adding nodes
    for par in root.traverse_parents(False):
        nodenum += 1
        node2num[par] = nodenum
        nodedict = node2dict(par)
        nodedict["root"] = False
        nodedict["type"] = str(type(par))
        G.add_node(node2num[par], **nodedict)

    # adding edges
    for par in root.traverse_parents(True):
        for child in par.children:
            if not G.has_edge(node2num[child], node2num[par]):
                G.add_edge(node2num[child], node2num[par])
                
    return G


def write2file(node):
    G = node2nxgraph(node)
    data = json_graph.node_link_data(G)
    data["topsort"] = list(nx.topological_sort(G))
    n = len(data["topsort"])
    #temp_filename = tempfile.mktemp()
    import os
    from pathlib import Path
    dest_dir = Path(os.environ['HOME']) / '.cache' / 'fhecompiler'
    dest_dir.mkdir(parents=True, exist_ok=True)
    temp_filename = dest_dir / "test.json"
    serialized_data = json.dumps(data, indent=2, default=lambda o: o.__dict__)
    with open(temp_filename, "w") as file:
        file.write(serialized_data)
    return temp_filename


In [4]:
#from compyler.json_serialization import write2file

#original_graph = deepcopy(node) ## temporary thing for plain_compute
node = b
path = write2file(node)